# Quantization aware (QKeras) DeepSet Permutation Invariant NN for Jet tagging using jet constituents from HLS data implemented by Patrick ( from paper https://arxiv.org/abs/1703.06114 )

##   Original code from: https://github.com/bb511/know_dist

## Author: Andre Sznajder

 

In [1]:
#!fusermount -u drive
#! pip install einops
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#data_dir = '/content/gdrive/My Drive/Colab Notebooks/Data/'

In [2]:
import tensorflow as tf
print(f"TensorFlow {tf.__version__}")
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print(f"Number of available GPUs : {len(gpus)}")
    tf.config.set_visible_devices(gpus[0],"GPU")
    tf.config.experimental.set_memory_growth(gpus[0],True)
else:
    print("No GPU available, using CPU !!!")    


TensorFlow 2.8.0
Number of available GPUs : 1


## Load the train and test data as numpy arrays

In [3]:
import numpy as np
import h5py
import os

#Data PATH
DATA_PATH = '/Users/sznajder/WorkM1/workdir/data/'

nconstit = 32

X_train_val = np.load("../../data/X_train_val_nconst_{}.npy".format(nconstit))
X_test = np.load("../../data/X_test_nconst_{}.npy".format(nconstit))
Y_train_val = np.load("../../data/Y_train_val_nconst_{}.npy".format(nconstit))
Y_test = np.load("../../data/Y_test_nconst_{}.npy".format(nconstit))

print("Loaded X_train_val ----> shape:", X_train_val.shape)
print("Loaded X_test      ----> shape:", X_test.shape)
print("Loaded Y_train_val ----> shape:", X_train_val.shape)
print("Loaded Y_test      ----> shape:", X_test.shape)

nfeat = X_train_val.shape[-1]

Loaded X_train_val ----> shape: (589600, 32, 3)
Loaded X_test      ----> shape: (290400, 32, 3)
Loaded Y_train_val ----> shape: (589600, 32, 3)
Loaded Y_test      ----> shape: (290400, 32, 3)


## Define DeepSet Permutation Equivariant Layer

In [4]:
# baseline keras model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam, Nadam
from tensorflow.keras.layers import BatchNormalization, Input, Activation, Dense, Conv1D, Add, RepeatVector
from tensorflow.keras.layers import Flatten, Reshape, GlobalAveragePooling1D, Concatenate, UpSampling1D, AveragePooling1D, MaxPooling1D  
from tensorflow.keras import utils
from qkeras import *

import tensorflow.keras.layers as KL

class PermutationEquivariantMax(KL.Layer):
    """Permutation equivariant neural network layer with max operation."""

    def __init__(self, dim):
        super(PermutationEquivariantMax, self).__init__()
        self.gamma = KL.Dense(dim)
        self.lambd = KL.Dense(dim, use_bias=False)

    def call(self, inputs: np.ndarray, **kwargs):
        x_maximum = tf.reduce_max(inputs, axis=1, keepdims=True)
        x_maximum = self.lambd(x_maximum)
        x = self.gamma(inputs)
        x = x - x_maximum

        return x


class PermutationEquivariantMean(KL.Layer):
    """Permutation equivariant neural network layer with mean operation."""

    def __init__(self, dim, kernel_quantizer, bias_quantizer):
        super(PermutationEquivariantMean, self).__init__()
        self.gamma = QDense(dim, kernel_quantizer, bias_quantizer, name='dense_1' )          # multiply features by weights and add bias, sharing the weights and bias. Returns #dim embeded features per element 
        self.lambd = QDense(dim, kernel_quantizer, name='densenobias_1',use_bias=False)      # multiply features by weights, sharing the weights over elements. Returns #dim embeded features per element
        self.dim = dim
        
    def call(self, inputs: np.ndarray, **kwargs):
        x_mean = GlobalAveragePooling1D(name='avgpool_1')(inputs) # returns a tensor of size (#elements,#features) containing the mean of each individual feature over elements 
        x_mean = Reshape((1,-1),name='reshap')(x_mean)      # reshape tensor to original 3D format (batch, 1, nfeat)
#        x_mean = UpSampling1D(size=self.dim,name='upsampl_1')(x_mean)    # make #(nconstit) copies of tensor along axis=1

        x_mean = self.lambd(x_mean)   # multiply each mean feature by a weights returning #dim embeded features per element (λI)
        x = self.gamma(inputs)        # returns #dim embeded features per element 
        x = x - x_mean                # get deviation of X from mean features (λI) 

        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "dim": self.dim,
                "kernel_quantizer": kernel_quantizer,
                "bias_quantizer": bias_quantizer,
            }
        )
        return config



## Define DeepSet Permutation Equivariant Model

In [5]:
#########################################################################################################
'''
# Silence the info from tensorflow in which it brags that it can run on cpu nicely.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
keras.utils.set_random_seed(123)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

from util.terminal_colors import tcols
from . import util as dsutil

tf.keras.backend.set_floatx("float64")

util.util.device_info()
outdir = util.util.make_output_directory("trained_deepsets", args["outdir"])
util.util.save_hyperparameters_file(args, outdir)

data = Data.shuffled(**args["data_hyperparams"])
'''
#########################################################################################################

# Quantized bits
nbits=8
integ=0

#qbits = quantized_bits(nbits,integ,alpha=1.0)
#qact = 'quantized_relu('+str(nbits)+',0)'

# Set QKeras quantizer and activation 
if nbits == 1:
    qbits = 'binary(alpha=1)'
elif nbits == 2:
    qbits = 'ternary(alpha=1)'
else:
    qbits = 'quantized_bits({},0,alpha=1)'.format(nbits)

qact = 'quantized_relu({},0)'.format(nbits)

# Print
print("Training with max # of contituents = ", nconstit)
print("Number of node features = ", nfeat)
print("Quantization with nbits=",nbits)
print("Quantization of integer part=",integ)

#############################################################################


nnodes_phi = nconstit
nnodes_rho = nconstit/2
activ      = "relu"
#activ      = "elu"

# Number of target classes
nclasses = len(Y_train_val[0]) 

# Instantiate Tensorflow input tensors in Batch mode 
inp = Input(shape=(nconstit,nfeat), name="inp")   # Conv1D input format

# Input point features BatchNormalization 
h = BatchNormalization(name='BatchNorm')(inp)

# Phi MLP ( permutation equivariant layers )
h = PermutationEquivariantMean(nnodes_phi, kernel_quantizer=qbits, bias_quantizer=qbits )(h)
h = QActivation(qact,name='qrelu_phi1')(h)
h = PermutationEquivariantMean(nnodes_phi, kernel_quantizer=qbits, bias_quantizer=qbits )(h)
h = QActivation(qact,name='qrelu_phi2')(h)
h = PermutationEquivariantMean(nnodes_phi, kernel_quantizer=qbits, bias_quantizer=qbits )(h)
phi_out = QActivation(qact,name='qrelu_phi3')(h)
 
# Agregate features (taking mean) over set elements  
mean = GlobalAveragePooling1D(name='avgpool')(phi_out)      # return mean of features over elements
#mean = tf.reduce_mean(phi_out, axis=1)                     # return mean of features over elements
 
# Rho MLP
h = QDense(nnodes_rho, kernel_quantizer=qbits, bias_quantizer=qbits,name='qdense_rho1' )(mean)
h = QActivation(qact,name='qrelu_rho1')(h)
h = QDense(nnodes_rho, kernel_quantizer=qbits, bias_quantizer=qbits,name='qdense_rho2' )(h)
h = QActivation(qact,name='qrelu_rho2')(h)
h = QDense(nclasses, kernel_quantizer=qbits, bias_quantizer=qbits,name='qdense_rho3' )(h)
out = Activation('softmax',name='softmax')(h)

# Build the model
arch = 'QDeepSets_PermutationInv'
model = Model(inputs=inp, outputs=out)

# Define the optimizer ( minimization algorithm )
#optim = SGD(learning_rate=0.0001,decay=1e-6)
#optim = Adam(learning_rate=0.0001)
optim = Adam()

# compile the model
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# print the model summary
model.summary()




Training with max # of contituents =  32
Number of node features =  3
Quantization with nbits= 8
Quantization of integer part= 0
Metal device set to: Apple M1


2023-05-06 13:55:54.070662: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-06 13:55:54.070951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inp (InputLayer)            [(None, 32, 3)]           0         
                                                                 
 BatchNorm (BatchNormalizati  (None, 32, 3)            12        
 on)                                                             
                                                                 
 permutation_equivariant_mea  (None, 32, 32)           224       
 n (PermutationEquivariantMe                                     
 an)                                                             
                                                                 
 qrelu_phi1 (QActivation)    (None, 32, 32)            0         
                                                                 
 permutation_equivariant_mea  (None, 32, 32)           2080      
 n_1 (PermutationEquivariant                                 

## Run training

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

# early stopping callback
es = EarlyStopping(monitor='val_categorical_accuracy', patience=10)

# Learning rate scheduler 
ls = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.2, patience=10)

# model checkpoint callback
# this saves our model architecture + parameters into mlp_model.h5
chkp = ModelCheckpoint('QDeepSet_model.h5', monitor='val_categorical_accuracy', 
                                   verbose=0, save_best_only=True, 
                                   save_weights_only=False, mode='auto', 
                                   save_freq=1)

#tb = TensorBoard("/Users/sznajder/WorkM1/miniforge3/tensorflow_macos/arm64/workdir/logs")

# Train classifier
history = model.fit( X_train_val, Y_train_val, 
                    epochs=20, 
                    batch_size=512, 
                    verbose=1,
                    callbacks=[es,ls], 
                    validation_split=0.3 )
                    
#                    callbacks=[early_stopping, model_checkpoint], 
 
#
# Set NN and output name
fname = arch+'_nconst_'+str(nconstit)+'_nbits_'+str(nbits)
print('Saving Model : ',fname)


## Save the model+ weights
model.save('model_'+fname+'.h5')

## Save the model weights in a separate file
model.save_weights('weights_'+fname+'.h5')

Epoch 1/20


2023-05-06 13:55:55.629158: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-06 13:55:56.521064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


807/807 [==============================] - ETA: 0s - loss: 1.1431 - categorical_accuracy: 0.5493

2023-05-06 13:56:28.139872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


807/807 [==============================] - 42s 47ms/step - loss: 1.1431 - categorical_accuracy: 0.5493 - val_loss: 0.9445 - val_categorical_accuracy: 0.6467 - lr: 0.0010
Epoch 2/20
807/807 [==============================] - 37s 46ms/step - loss: 0.9074 - categorical_accuracy: 0.6599 - val_loss: 0.8779 - val_categorical_accuracy: 0.6764 - lr: 0.0010
Epoch 3/20
807/807 [==============================] - 37s 46ms/step - loss: 0.8664 - categorical_accuracy: 0.6771 - val_loss: 0.8543 - val_categorical_accuracy: 0.6867 - lr: 0.0010
Epoch 4/20
807/807 [==============================] - 37s 46ms/step - loss: 0.8416 - categorical_accuracy: 0.6880 - val_loss: 0.8208 - val_categorical_accuracy: 0.6984 - lr: 0.0010
Epoch 5/20
807/807 [==============================] - 37s 46ms/step - loss: 0.8324 - categorical_accuracy: 0.6911 - val_loss: 0.8316 - val_categorical_accuracy: 0.6893 - lr: 0.0010
Epoch 6/20
807/807 [==============================] - 37s 46ms/step - loss: 0.8227 - categorical_accuracy:

NameError: name 'nmax' is not defined

## Plot performance
Here, we plot the history of the training and the performance in a ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

#plt.rcParams['axes.unicode_minus'] = False

# Plot loss vs epoch
plt.figure(figsize=(15,10))
ax = plt.subplot(2, 2, 1)
ax.plot(history.history['loss'], label='loss')
ax.plot(history.history['val_loss'], label='val loss')
ax.legend(loc="upper right")
ax.set_xlabel('epoch')
ax.set_ylabel('loss')

# Plot accuracy vs epoch
ax = plt.subplot(2, 2, 2)
#ax.plot(history.history['accuracy'], label='accuracy')
#ax.plot(history.history['val_accuracy'], label='val accuracy')
ax.plot(history.history['categorical_accuracy'], label='categorical_accuracy')
ax.plot(history.history['val_categorical_accuracy'], label='val categorical accuracy')
ax.legend(loc="upper left")
ax.set_xlabel('epoch')
ax.set_ylabel('acc')

# Plot the ROC curves
labels = ['gluon', 'quark', 'W', 'Z', 'top']
fpr = {}
tpr = {}
auc1 = {}
precision = {}
recall = {}
NN = {}
NP = {}
TP = {}
FP = {}
TN = {}
FN = {}
tresholds = {}


ax = plt.subplot(2, 2, 3)
Y_predict = model.predict(X_test)


# Loop over classes(labels) to get metrics per class
for i, label in enumerate(labels):
    fpr[label], tpr[label], tresholds[label] = roc_curve(Y_test[:,i], Y_predict[:,i])
#    precision[label], recall[label], tresholds = precision_recall_curve(Y_test[:,i], Y_predict[:,i]) 
    print( np.unique(Y_test[:,i], return_counts=True) )
    _ , N = np.unique(Y_test[:,i], return_counts=True) # count the NEGATIVES and POSITIVES samples in your test set
    NN[label] = N[0]                   # number of NEGATIVES 
    NP[label] = N[1]                   # number of POSITIVES
    TP[label] = tpr[label]*NP[label]
    FP[label] = fpr[label]*NN[label] 
    TN[label] = NN[label] - FP[label]
    FN[label] = NP[label] - TP[label]

    auc1[label] = auc(fpr[label], tpr[label])
    ax.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))

ax.semilogy()
ax.set_xlabel("sig. efficiency")
ax.set_ylabel("bkg. mistag rate")
ax.set_ylim(0.001,1)
#ax.set_grid(True)
ax.legend(loc='lower right')
#plt.savefig('%s/ROC.pdf'%(options.outputDir))



# Plot DNN output 
ax = plt.subplot(2, 2, 4)
X = np.linspace(0.0, 1.0, 20)
hist={}
for i, name in enumerate(labels):
    hist[name] = ax.hist(Y_predict, bins=X, label=name ,histtype='step')
ax.semilogy()
ax.set_xlabel('DNN Output')
ax.legend(prop={'size': 10})
ax.legend(loc='lower left')


# Display plots
fig = plt.gcf()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Save plots
fig.savefig(fname+'.pdf')


# Save FPR for a given TPR value ( 30% , 50% & 80%)
with open('FPR@TPR_'+fname+'.csv', 'w') as file:
  file.write("model,label,treshold,tpr,fpr\n")
  for label in labels:
    for t in [0.3, 0.5, 0.8]:
      index = np.argmax(tpr[label]>t)
      file.write( arch+','+label+','+str(t)+','+str(tpr[label][index])+','+str(fpr[label][index])+'\n' )
      print("Label = ", label , " with treshold = ",t)
      print("TPR = ",tpr[label][index])
      print("FPR = ",fpr[label][index])
      print(" ")
               
               
# Save ROC AUC for each label
with open('ROCAUC_'+fname+'.csv', 'w') as file:
  header = labels[0]+', '+labels[1]+', '+labels[2]+', '+labels[3]+', '+labels[4]+'\n'
  file.write(header)
  rocauc = str(auc1[labels[0]])+', '+str(auc1[labels[1]])+', '+str(auc1[labels[2]])+', '+str(auc1[labels[3]])+', '+str(auc1[labels[4]])
  file.write(rocauc)



# Save NN Accuracy for treshold of 0.5 for each label and the average over all classes
acc_avg = float(accuracy_score (np.argmax(Y_test,axis=1), np.argmax(Y_predict,axis=1)))
with open('ACCURACY_'+fname+'.csv', 'w') as file:
  header = labels[0]+', '+labels[1]+', '+labels[2]+', '+labels[3]+', '+labels[4]+', '+'acc_avg'+'\n'
  file.write(header)
  accuracy = ''
  for label in labels:  
    idx = np.argmax( tresholds[label] <= 0.5 )
    accuracy += str( (TP[label][idx]+TN[label][idx])/(NP[label]+NN[label]) )+', '
  accuracy += str(acc_avg) 
  file.write(accuracy)


'''
# Save confusion matrix ndarrays to .npz file
with open('CONF_MATRIX_'+fname+'.npz', 'wb') as file:
    vars = {}
    vars[arch]=np.array(1) # save model name
    for label in labels:
        vars['tresholds_'+label+'_'+arch] = tresholds[label]
        vars['TP_'+label+'_'+arch] = TP[label]
        vars['FP_'+label+'_'+arch] = FP[label]
        vars['TN_'+label+'_'+arch] = TN[label]
        vars['FN_'+label+'_'+arch] = FN[label]
        vars['TPR_'+arch] = tpr[label]
        vars['FPR_'+arch] = fpr[label]
        vars['NP_'+arch]= NP[label]
        vars['NN_'+arch]= NN[label]
        vars['auc_'+arch] = auc1[label] 
#        print(vars)
    np.savez(file, **vars)
'''

'''
# Save a sample of events for HLS
njets=3000
print(X_test.shape)
np.save('x_test.npy', X_test[0:njets,:])
np.save('y_test.npy', Y_test[0:njets,:])
'''